In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]

In [2]:
alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])

In [3]:
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [4]:
with open('query_dict/id_querry_clean.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)

In [5]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [6]:
vals = all_groups['DocumentId'].values

In [7]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)
with open("url_simple.pickle", 'rb') as file:
    url_simple = pickle.load(file)
with open("host_simple.pickle", 'rb') as file:
    host_simple = pickle.load(file)

In [8]:
#self.show_poses = np.zeros((10,),dtype='int')
#self.show_poses_cl = np.zeros((10,),dtype='int')
#self.click_poses = np.zeros((10,),dtype='int')

In [9]:
template = np.arange(1,11)
click_feats = dict()
click_feats1 = dict()
click_feats2 = dict()
click_feats3 = dict()
click_feats4 = dict()
click_feats['url_fctr'] = []
click_feats['url_lctr'] = []
click_feats['url_avg_show_poses'] = []
click_feats['url_avg_show_poses_cl'] = []
click_feats['url_avg_click_poses'] = []
click_feats['url_show_poses_cl1'] = []
click_feats['url_show_poses_cl2'] = []
click_feats['url_show_poses_cl3'] = []
click_feats['url_show_poses_cl4'] = []
click_feats['url_show_poses_cl5'] = []
click_feats['url_show_poses_cl6'] = []
click_feats['url_show_poses_cl7'] = []
click_feats['url_show_poses_cl8'] = []
click_feats['url_show_poses_cl9'] = []
click_feats['url_show_poses_cl10'] = []

click_feats2['url_prob_cc1'] = []
click_feats2['url_prob_cc2'] = []
click_feats2['url_prob_cc3'] = []
click_feats2['url_prob_cc4'] = []
click_feats2['url_prob_cc5'] = []
click_feats2['url_prob_cc6'] = []
click_feats2['url_prob_cc7'] = []
click_feats2['url_prob_cc8'] = []
click_feats2['url_prob_cc9'] = []
click_feats2['url_prob_cc10'] = []

click_feats['host_fctr'] = []
click_feats['host_lctr'] = []
click_feats['host_avg_show_poses'] = []
click_feats['host_avg_show_poses_cl'] = []
click_feats['host_avg_click_poses'] = []
click_feats1['host_show_poses_cl1'] = []
click_feats1['host_show_poses_cl2'] = []
click_feats1['host_show_poses_cl3'] = []
click_feats1['host_show_poses_cl4'] = []
click_feats1['host_show_poses_cl5'] = []
click_feats1['host_show_poses_cl6'] = []
click_feats1['host_show_poses_cl7'] = []
click_feats1['host_show_poses_cl8'] = []
click_feats1['host_show_poses_cl9'] = []
click_feats1['host_show_poses_cl10'] = []

click_feats2['host_prob_cc1'] = []
click_feats2['host_prob_cc2'] = []
click_feats2['host_prob_cc3'] = []
click_feats2['host_prob_cc4'] = []
click_feats2['host_prob_cc5'] = []
click_feats2['host_prob_cc6'] = []
click_feats2['host_prob_cc7'] = []
click_feats2['host_prob_cc8'] = []
click_feats2['host_prob_cc9'] = []
click_feats2['host_prob_cc10'] = []
for i in range(1,11):
    click_feats3['url_prob_sc{}'.format(i)] = []
    click_feats3['host_prob_sc{}'.format(i)] = []
for i in range(1,11):
    click_feats4['url_prob_ss{}'.format(i)] = []
    click_feats4['host_prob_ss{}'.format(i)] = []

In [10]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    click_feats['url_fctr'].append(url_simple[key1].num_first_clicks/(url_simple[key1].total_shows+0.0000001))
    click_feats['url_lctr'].append(url_simple[key1].num_last_clicks/(url_simple[key1].total_shows+0.0000001))
    click_feats['url_avg_show_poses'].append(np.sum(template*url_simple[key1].show_poses)/(np.sum(url_simple[key1].show_poses)+0.0000001))
    click_feats['url_avg_show_poses_cl'].append(np.sum(template*url_simple[key1].show_poses_cl)/(np.sum(url_simple[key1].show_poses_cl)+0.0000001))
    click_feats['url_avg_click_poses'].append(np.sum(template*url_simple[key1].click_poses)/(np.sum(url_simple[key1].click_poses)+0.0000001))
    click_feats['url_show_poses_cl1'].append(url_simple[key1].show_poses_cl[0])
    click_feats['url_show_poses_cl2'].append(url_simple[key1].show_poses_cl[1])
    click_feats['url_show_poses_cl3'].append(url_simple[key1].show_poses_cl[2])
    click_feats['url_show_poses_cl4'].append(url_simple[key1].show_poses_cl[3])
    click_feats['url_show_poses_cl5'].append(url_simple[key1].show_poses_cl[4])
    click_feats['url_show_poses_cl6'].append(url_simple[key1].show_poses_cl[5])
    click_feats['url_show_poses_cl7'].append(url_simple[key1].show_poses_cl[6])
    click_feats['url_show_poses_cl8'].append(url_simple[key1].show_poses_cl[7])
    click_feats['url_show_poses_cl9'].append(url_simple[key1].show_poses_cl[7])
    click_feats['url_show_poses_cl10'].append(url_simple[key1].show_poses_cl[9])
    click_feats2['url_prob_cc1'].append(url_simple[key1].click_poses[0]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc2'].append(url_simple[key1].click_poses[1]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc3'].append(url_simple[key1].click_poses[2]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc4'].append(url_simple[key1].click_poses[3]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc5'].append(url_simple[key1].click_poses[4]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc6'].append(url_simple[key1].click_poses[5]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc7'].append(url_simple[key1].click_poses[6]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc8'].append(url_simple[key1].click_poses[7]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc9'].append(url_simple[key1].click_poses[8]/(url_simple[key1].total_clicks+0.0000001))
    click_feats2['url_prob_cc10'].append(url_simple[key1].click_poses[9]/(url_simple[key1].total_clicks+0.0000001))
    for i in range(1,11):
        click_feats3['url_prob_sc{}'.format(i)].append(url_simple[key1].show_poses_cl[i-1]/(url_simple[key1].total_clicks+0.0000001))
    for i in range(1,11):
        click_feats4['url_prob_ss{}'.format(i)].append(url_simple[key1].show_poses[i-1]/(url_simple[key1].total_shows+0.0000001))

100%|██████████| 606050/606050 [02:05<00:00, 4836.67it/s]


In [11]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    click_feats['host_fctr'].append(host_simple[key1].num_first_clicks/(host_simple[key1].total_shows+0.0000001))
    click_feats['host_lctr'].append(host_simple[key1].num_last_clicks/(host_simple[key1].total_shows+0.0000001))
    click_feats['host_avg_show_poses'].append(np.sum(template*host_simple[key1].show_poses)/(np.sum(host_simple[key1].show_poses)+0.0000001))
    click_feats['host_avg_show_poses_cl'].append(np.sum(template*host_simple[key1].show_poses_cl)/(np.sum(host_simple[key1].show_poses_cl)+0.0000001))
    click_feats['host_avg_click_poses'].append(np.sum(template*host_simple[key1].click_poses)/(np.sum(host_simple[key1].click_poses)+0.0000001))
    click_feats1['host_show_poses_cl1'].append(host_simple[key1].show_poses_cl[0])
    click_feats1['host_show_poses_cl2'].append(host_simple[key1].show_poses_cl[1])
    click_feats1['host_show_poses_cl3'].append(host_simple[key1].show_poses_cl[2])
    click_feats1['host_show_poses_cl4'].append(host_simple[key1].show_poses_cl[3])
    click_feats1['host_show_poses_cl5'].append(host_simple[key1].show_poses_cl[4])
    click_feats1['host_show_poses_cl6'].append(host_simple[key1].show_poses_cl[5])
    click_feats1['host_show_poses_cl7'].append(host_simple[key1].show_poses_cl[6])
    click_feats1['host_show_poses_cl8'].append(host_simple[key1].show_poses_cl[7])
    click_feats1['host_show_poses_cl9'].append(host_simple[key1].show_poses_cl[7])
    click_feats1['host_show_poses_cl10'].append(host_simple[key1].show_poses_cl[9])
    click_feats2['host_prob_cc1'].append(host_simple[key1].click_poses[0]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc2'].append(host_simple[key1].click_poses[1]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc3'].append(host_simple[key1].click_poses[2]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc4'].append(host_simple[key1].click_poses[3]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc5'].append(host_simple[key1].click_poses[4]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc6'].append(host_simple[key1].click_poses[5]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc7'].append(host_simple[key1].click_poses[6]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc8'].append(host_simple[key1].click_poses[7]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc9'].append(host_simple[key1].click_poses[8]/(host_simple[key1].total_clicks+0.0000001))
    click_feats2['host_prob_cc10'].append(host_simple[key1].click_poses[9]/(host_simple[key1].total_clicks+0.0000001))
    for i in range(1,11):
        click_feats3['host_prob_sc{}'.format(i)].append(host_simple[key1].show_poses_cl[i-1]/(host_simple[key1].total_clicks+0.0000001))
    for i in range(1,11):
        click_feats4['host_prob_ss{}'.format(i)].append(host_simple[key1].show_poses[i-1]/(host_simple[key1].total_shows+0.0000001))

100%|██████████| 606050/606050 [01:53<00:00, 5341.04it/s]


In [12]:
with open("click_feats_advanced3.pickle", 'wb') as file:
    pickle.dump(click_feats, file)

In [13]:
with open("click_feats_advanced4.pickle", 'wb') as file:
    pickle.dump(click_feats1, file)

In [14]:
with open("click_feats_advanced5.pickle", 'wb') as file:
    pickle.dump(click_feats2, file)

In [15]:
with open("click_feats_advanced6.pickle", 'wb') as file:
    pickle.dump(click_feats3, file)

In [16]:
with open("click_feats_advanced7.pickle", 'wb') as file:
    pickle.dump(click_feats4, file)